# 12.Seq2seq으로 번역기 만들기[프로젝트]

---

## 12-1. 프로젝트: 한영 번역기 만들기

### **라이브러리 버전을 확인해 봅니다**

---

사용할 라이브러리 버전을 둘러봅시다.

In [1]:
import pandas
import tensorflow
import matplotlib

print(pandas.__version__)
print(tensorflow.__version__)
print(matplotlib.__version__)

1.3.3
2.6.0
3.4.3


#### Step 1. 데이터 다운로드

---

아래 링크에서 `korean-english-park.train.tar.gz`를 다운로드받아 한영 병렬 데이터를 확보합니다.
- [jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)

---

`~/aiffel/going_deeper_data/korean-english-park.train.tar.gz` 파일 구축 완료  

- 압축 해제 완료
    - `~/aiffel/going_deeper_data/korean-english-park.train.en`
    - `~/aiffel/going_deeper_data/korean-english-park.train.ko`

#### Step 2. 데이터 정제

---

1. `set` 데이터형이 **중복을 허용하지 않는다는 것을 활용**해 중복된 데이터를 제거하도록 합니다. 데이터의 **병렬 쌍이 흐트러지지 않게 주의**하세요! 중복을 제거한 데이터를 `cleaned_corpus`에 저장합니다.
2. 앞서 정의한 `preprocessing()` 함수는 한글에서는 동작하지 않습니다. **한글에 적용할 수 있는 정규식**을 추가하여 함수를 재정의하세요!
3. 타겟 언어인 영문엔 `<start>` 토큰과 `<end>`토큰을 추가하고 `split()`함수를 이용하여 토큰화합니다. 한글 토큰화는 KoNLPy의 `mecab` 클래스를 사용합니다.

모든 데이터를 사용할 경우 학습에 굉장이 오랜 시간이 걸립니다. `cleaned_corpus`로부터 **토큰의 길이가 40 이하인 데이터를 선별**하여 `eng_corpus`와 `kor_corpus`를 각각 구축하세요.

---

*데이터 확인(file read)*

In [2]:
# 한글 데이터 읽기
with open('/aiffel/aiffel/going_deeper_data/korean-english-park.train.ko', "r") as f:
    ko_raw = f.read().splitlines()
    
print("Data Size:", len(ko_raw))
print("Example:")

for sen in ko_raw[0:100][::20]: print(">>", sen)

Data Size: 94123
Example:
>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>> 북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.
>> "경호 로보트가 침입자나 화재를 탐지하기 위해서 개인적으로, 그리고 전문적으로 사용되고 있습니다."
>> 수자원부 당국은 논란이 되고 있고, 막대한 비용이 드는 이 사업에 대해 내년에 건설을 시작할 계획이다.
>> 또한 근력 운동은 활발하게 걷는 것이나 최소한 20분 동안 뛰는 것과 같은 유산소 활동에서 얻는 운동 효과를 심장과 폐에 주지 않기 때문에, 연구학자들은 근력 운동이 심장에 큰 영향을 미치는지 여부에 대해 논쟁을 해왔다.


In [3]:
# 영어 데이터 읽기
with open('/aiffel/aiffel/going_deeper_data/korean-english-park.train.en', "r") as f:
    en_raw = f.read().splitlines()
    
print("Data Size:", len(en_raw))
print("Example:")

for sen in en_raw[0:100][::20]: print(">>", sen)

Data Size: 94123
Example:
>> Much of personal computing is about "can you top this?"
>> Amid mounting pressure on North Korea to abandon its nuclear weapons program Japanese and North Korean diplomats have resumed talks on normalizing diplomatic relations.
>> “Guard robots are used privately and professionally to detect intruders or fire,” Karlsson said.
>> Authorities from the Water Resources Ministry plan to begin construction next year on the controversial and hugely expensive project.
>> Researchers also have debated whether weight-training has a big impact on the heart, since it does not give the heart and lungs the kind of workout they get from aerobic activities such as brisk walking or running for at least 20 minutes.


---

*2. 앞서 정의한 `preprocessing()` 함수는 한글에서 동작하지 않습니다. **한글에 적용할 수 있는 정규식**을 추가하여 함수를 재정의하세요!*  
  
한글 문장 전처리 함수 &rarr; `preprocess_ko_sentence()`  
영어 문장 전처리 함수 &rarr; `preprocess_en_sentence()`

In [4]:
# 한글 문장 전처리 함수 선언
def preprocess_ko_sentence(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

print("preprocess_ko_sentence() 선언 완료!")

preprocess_ko_sentence() 선언 완료!


In [5]:
# 영어 문장 전처리 함수 선언
def preprocess_en_sentence(sentence, s_token=False, e_token=False):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()

    if s_token:
        sentence = '<start> ' + sentence

    if e_token:
        sentence += ' <end>'
    
    return sentence

print("preprocess_en_sentence() 선언 완료")

preprocess_en_sentence() 선언 완료


---

*1. `set` 데이터형이 **중복을 허용하지 않는다는 것을 활용**해 중복된 데이터를 제거하도록 합니다. 데이터의 **병렬 쌍이 흐트러지지 않게 주의**하세요! 중복을 제거한 데이터를 `cleaned_corpus`에 저장합니다. *

In [6]:
from tqdm import tqdm
import re

# 위에서 선언한 함수 실행 후 저장될 배열 선언
preprocessed_ko = []
preprocessed_en = []

# 한글 문장, 영어 문장 배열 동시에 읽어 들어가기
for (v1, v2) in tqdm(zip(ko_raw, en_raw), desc='read languages raw..'):
    # 한글 문장 전처리하면서 저장
    preprocessed_ko.append(preprocess_ko_sentence(v1))
    # 영어 문장 전처리하면서 저장
    preprocessed_en.append(preprocess_en_sentence(v2, s_token=True, e_token=True))

# set 타입 활용하여 중복된 데이터 제거 후 cleaned_corpus에 저장
cleaned_corpus = set(zip(preprocessed_ko, preprocessed_en))
# preprocessed_ko, preprocessed_en 길이들 확인
print(f"preprocessed_ko 전체 길이 : {len(preprocessed_ko)}")
print(f"preprocessed_en 전체 길이 : {len(preprocessed_en)}")
# cleaned_corpus 전체 길이 확인
print(f"cleaned_corpus 전체 길이 : {len(cleaned_corpus)}")
# # 데이터 확인
cnt = 0
for val in cleaned_corpus:
    print(val)
    cnt += 1
    
    if cnt > 4:
        break

read languages raw..: 94123it [00:03, 24531.23it/s]

preprocessed_ko 전체 길이 : 94123
preprocessed_en 전체 길이 : 94123
cleaned_corpus 전체 길이 : 78794
('국제 유가와 금가격도 큰 폭 으로 떨어졌다', '<start> oil and gold prices plunged . <end>')
('피트는 일 칸 국제영화제에서 처음으로 공개되는 이 영화의 프로듀서다', '<start> pitt is a producer on the movie , which had its world premiere at the cannes film festival on monday . <end>')
('이 소식통은 오클리가 일 녹스빌 연방수사국 사무실에 자진 출두 혐의를 인정했다고 말했다', '<start> he voluntarily surrendered thursday at an fbi field office in knoxville , the sources said . <end>')
('옥스포드 거리 상점은 모스의 특별 쇼핑 세션 참여로 변화를 맞았다', '<start> and so the masses were happy to wait hours in line outside topshop s oxford street store for the chance to attend a special shopping session of moss s inspirations . <end>')
('머큐리 에너지 직원이 일 전기를 끊기 위해 집으로 방문하기 전까지 뮬리아가는 달러 약 만 원 를 연체했다', '<start> six days before a mercury energy representative arrived tuesday at the house to disconnect the electricity , she was in arrears . <end>')


---

3. 타겟 언어인 영문엔 `<start>` 토큰과 `<end>`토큰을 추가하고 `split()`함수를 이용하여 토큰화합니다. 한글 토큰화는 KoNLPy의 `mecab` 클래스를 사용합니다.

In [7]:
# 토크나이징 함수 - 한국어
def tokenize_kr(sentence):
    import MeCab

    # MeCab 초기화
    m = MeCab.Tagger()
    
    # 형태소, 품사 등이 있는 문장들로 반환
    morphs = m.parse(sentence)
    
    # 형태소와 품사 정보를 출력
    lines = morphs.split('\n')
    
    # 특정 품사 단어들을 담을 토큰 리스트
    tokens = []
    
    for line in lines:
        if line == "EOS" or line == '':
            break
        else:
            morph_info = line.split('\t')
            if len(morph_info) >= 2:
                # 형태소
                word = morph_info[0]
                # 품사
                pos = morph_info[1].split(',')[0]
                # 품사가 명사 또는 고유명사인 경우 저장
                if pos in ["NNG", "NNP"]:
                    tokens.append(word)
    return tokens

In [8]:
# 토크나이징 함수 - 영어
def tokenize_en(sentence):
    # 영문 - <start> & <end> 토큰 추가 --> 이미 처리되어 들어옴
    # split()을 이용하여 토큰화
    tokens = sentence.split(' ')
    
    return tokens

---

모든 데이터를 사용할 경우 학습에 굉장이 오랜 시간이 걸립니다. `cleaned_corpus`로부터 **토큰의 길이가 40 이하인 데이터를 선별**하여 `eng_corpus`와 `kor_corpus`를 각각 구축하세요.

In [9]:
# 언어별 corpus
eng_corpus = []
kor_corpus = []

# 토큰 길이 제한
max_len = 40

# for loop
for val in tqdm(cleaned_corpus, desc='check kr & en corpus...'):
    # 문장 당 토큰 담을 리스트
    ko_sentence = []
    en_sentence = []
    
    # 한국어 case
    # 한국어 문장 하나 당 명사&고유명사 리스트 반환)
    ko_sentence = tokenize_kr(val[0])
    
    # 영어 case
    # 영어 문장 하나 당 단어 리스트 반환 (<start>, <end> 토큰 포함)
    en_sentence = tokenize_en(val[1])
    
    if len(ko_sentence) <= max_len:
        # 토큰 뭉치가 길이 40 이하일 때만 저장 (한국어 기준)
        kor_corpus.append(ko_sentence)
        eng_corpus.append(en_sentence)

check kr & en corpus...: 100%|██████████| 78794/78794 [01:44<00:00, 755.48it/s]


In [10]:
print(f"kor 길이 : {len(kor_corpus)}")
print(f"en 길이 : {len(eng_corpus)}")

kor 길이 : 78785
en 길이 : 78785


In [11]:
print(kor_corpus[0])
print(eng_corpus[0])

['국제', '유가', '금', '가격', '폭']
['<start>', 'oil', 'and', 'gold', 'prices', 'plunged', '.', '<end>']


#### Step 3. 데이터 토큰화

---

앞서 정의한 `tokenize()` 함수를 사용해 데이터를 텐서로 변환하고 각각의 `tokenizer`를 얻으세요! 단어의 수는 실험을 통해 적당한 값을 맞춰주도록 합니다.! (최소 10,000 이상!)

> *주의: 난이도에 비해 데이터가 많지 않아 훈련 데이터와 검증 데이터를 따로 나누지는 않습니다.*

In [12]:
# tokenize() 함수 선언
def tokenize(corpus):
    # 단어 빈도 수 상위 1만개
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=10000, filters='', oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

**강의처럼 3만개를 기준으로 하지 않고 전체 기준에서 데이터 unique만 신경 써서 7만개가 넘음**  
그래서 `train`과 `test`로 나눠봄

In [13]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 토큰화하기
enc_tensor, enc_tokenizer = tokenize(kor_corpus)
dec_tensor, dec_tokenizer = tokenize(eng_corpus)
# train_test_split을 활용해서 훈련 데이터와 검증 데이터로 분리하기
enc_train, enc_test, dec_train, dec_test = train_test_split(enc_tensor, dec_tensor, test_size=0.2)

print(enc_tokenizer)
print(kor_corpus[0])

['국제', '유가', '금', '가격', '폭']


#### Step 4. 모델 설계

---

한국어를 영어로 잘 번역해 줄 멋진 **Attention 기반 Seq2seq 모델을 설계**하세요! 앞서 만든 모델에 `Dropout` 모듈을 추가하면 성능이 더 좋아집니다! `Embedding Size`와 `Hidden Size`는 실험을 통해 적당한 값을 맞춰 주도록 합니다!

In [14]:
# 강의 모델을 그대로 따라가보자
## 1. Bahdanau Attention
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.w_dec = tf.keras.layers.Dense(units)
        self.w_enc = tf.keras.layers.Dense(units)
        self.w_com = tf.keras.layers.Dense(1)
    
    def call(self, h_enc, h_dec):
        # h_enc shape: [batch x length x units]
        # h_dec shape: [batch x units]

        h_enc = self.w_enc(h_enc)
        h_dec = tf.expand_dims(h_dec, 1)
        h_dec = self.w_dec(h_dec)

        score = self.w_com(tf.nn.tanh(h_dec + h_enc))
        
        attn = tf.nn.softmax(score, axis=1)

        context_vec = attn * h_enc
        context_vec = tf.reduce_sum(context_vec, axis=1)

        return context_vec, attn

print("슝~")

슝~


In [15]:
## 2. Encoder
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        # TODO: Awesome Encoder Modules
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(enc_units,
                                       return_sequences=True)
		
    def call(self, x):
        # TODO: Awesome Process
        out = self.embedding(x)
        out = self.gru(out)
        
        return out

In [16]:
## 3. Decoder
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        # TODO: Awesome Decoder Modules
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)   # Attention 필수 사용!
    
    def call(self, x, h_dec, enc_out):
        # TODO: Awesome Process
        context_vec, attn = self.attention(enc_out, h_dec)

        out = self.embedding(x)
        out = tf.concat([tf.expand_dims(context_vec, 1), out], axis=-1)
        
        out, h_dec = self.gru(out)
        out = tf.reshape(out, (-1, out.shape[2]))
        out = self.fc(out)

        return out, h_dec, attn

In [21]:
# 코드를 실행하세요.
## model.summary()
### original BATCH_SIZE is 64
BATCH_SIZE     = 32
SRC_VOCAB_SIZE = len(enc_tokenizer.index_word) + 1
TGT_VOCAB_SIZE = len(dec_tokenizer.index_word) + 1

units         = 1024
embedding_dim = 512

encoder = Encoder(SRC_VOCAB_SIZE, embedding_dim, units)
decoder = Decoder(TGT_VOCAB_SIZE, embedding_dim, units)

# sample input
sequence_len = 30

sample_enc = tf.random.uniform((BATCH_SIZE, sequence_len))
sample_output = encoder(sample_enc)

print ('Encoder Output:', sample_output.shape)

sample_state = tf.random.uniform((BATCH_SIZE, units))

sample_logits, h_dec, attn = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                     sample_state, sample_output)

print ('Decoder Output:', sample_logits.shape)
print ('Decoder Hidden State:', h_dec.shape)
print ('Attention:', attn.shape)

del sample_output
del sample_logits
del h_dec
del attn

Encoder Output: (32, 30, 1024)
Decoder Output: (32, 44565)
Decoder Hidden State: (32, 1024)
Attention: (32, 30, 1)


In [22]:
## optimizier & loss 구현
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    
    return tf.reduce_mean(loss)

print("슝~")

슝~


In [23]:
## train step 구현하기
@tf.function
def train_step(src, tgt, encoder, decoder, optimizer, dec_tok):
    bsz = src.shape[0]
    loss = 0

    with tf.GradientTape() as tape:
        enc_out = encoder(src)
        h_dec = enc_out[:, -1]
        
        dec_src = tf.expand_dims([dec_tok.word_index['<start>']] * bsz, 1)

        for t in range(1, tgt.shape[1]):
            pred, h_dec, _ = decoder(dec_src, h_dec, enc_out)

            loss += loss_function(tgt[:, t], pred)
            dec_src = tf.expand_dims(tgt[:, t], 1)
        
    batch_loss = (loss / int(tgt.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

print("슝~")

슝~


강의 노드에서 이야기 하는  
`train_step()`의 학습 과정

---

1. Encoder에 소스 문장을 전달해 **컨텍스트 벡터**인 `enc_out` 을 생성
2. **t=0일 때,** Decoder의 Hidden State는 **Encoder의 Final State로 정의**. `h_dec = enc_out[:, -1]`
3. Decoder에 입력으로 전달할 `<start>` **토큰 문장 생성**
4. `<start>` 문장과 enc_out, Hidden State를 기반으로 다음 단어(t=1)를 예측. `pred`
5. 예측된 단어와 정답 간의 Loss를 구한 후, t=1의 **정답 단어를 다음 입력으로 사용** (예측 단어 X)
6. **반복!**

#### Step 5. 훈련하기

---

훈련엔 위에서 사용한 코드를 그대로 사용하되, `eval_step()` 부분이 없음에 유의합니다! 매 스탭 아래의 예문에 대한 번역을 생성하여 **본인이 생각하기에 가장 멋지게 번역한 Case를 제출**하세요! (Attention Map을 시각화해보는 것도 재밌을 거에요!)

> 참고: 데이터의 난이도가 높은 편이므로 생각만큼 결과가 잘 안나올 수 있습니다.

```shell
## 예문 ##
K1) 오바마는 대통령이다.
K2) 시민들은 도시 속에 산다.
K3) 커피는 필요 없다.
K4) 일곱 명의 사망자가 발생했다.

## 제출 ##
E1) obama is the president . <end>
E2) people are victims of the city . <end>
E2) the price is not enough . <end>
E2) seven people have died . <end>
```

In [24]:
# 모델 학습 #
from tqdm import tqdm    # tqdm
import random

EPOCHS = 10


for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)    # tqdm

    for (batch, idx) in enumerate(t):
        batch_loss = train_step(enc_train[idx:idx+BATCH_SIZE],
                                dec_train[idx:idx+BATCH_SIZE],
                                encoder,
                                decoder,
                                optimizer,
                                dec_tokenizer)
    
        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))    # tqdm
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))    # tqdm

  0%|          | 0/1970 [01:54<?, ?it/s]


ResourceExhaustedError:    OOM when allocating tensor with shape[1024,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node transpose_4}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[decoder_1/gru_3/PartitionedCall_48]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_step_364254]

Function call stack:
train_step -> train_step -> train_step


In [ ]:
def evaluate(sentence, encoder, decoder):
    attention = np.zeros((dec_train.shape[-1], enc_train.shape[-1]))
    
    sentence = preprocess_sentence(sentence)
    inputs = enc_tokenizer.texts_to_sequences([sentence.split()])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    result = ''

    enc_out = encoder(inputs)

    print(dec_tokenizer)

    dec_hidden = enc_out[:, -1]
    dec_input = tf.expand_dims([dec_tokenizer.word_index['<start>']], 0)

    for t in range(dec_train.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention[t] = attention_weights.numpy()

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0]).numpy()

        result += dec_tokenizer.index_word[predicted_id] + ' '

        if dec_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence, attention

        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention


def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def translate(sentence, encoder, decoder):
    result, sentence, attention = evaluate(sentence, encoder, decoder)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))
    
    attention = attention[:len(result.split()), :len(sentence.split())]
    plot_attention(attention, sentence.split(), result.split(' '))


translate("커피 좀 줄래요?", encoder, decoder)

**루브릭**  
아래의 기준을 바탕으로 프로젝트를 평가합니다.

|**평가문항**|**상세기준**|
|---|---|
|1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.|구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.|
|2. Attentional Seq2seq 모델이 정상적으로 구동된다.|seq2seq2 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.|
|3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.|테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.|

# 개인 회고

---

과정 확인을 해보면서 코드 수정해보고 싶었는데 OOM 에러가 훈련과정에서 떠서 아쉽 ㅠ